# pinkfish-challenge

Buy on the close on the SAME day a new 20 day high is set

In [1]:
# use future imports for python 3.x forward compatibility
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division
from __future__ import absolute_import

# other imports
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from talib.abstract import *

# project imports
import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [2]:
symbol = '^GSPC'
#symbol = 'SPY'
capital = 10000
start = datetime.datetime(2016, 1, 1)
end = datetime.datetime.now()

Define high trade periods

In [3]:
period = 20

Define Strategy Class

In [4]:
class Strategy(object):

    def __init__(self, symbol, capital, start, end, period):
        self._symbol = symbol
        self._capital = capital
        self._start = start
        self._end = end
        self._period = period

    def _algo(self):
        self._tlog.cash = self._capital
        start_flag = True
        end_flag = False

        for i, row in enumerate(self._ts.itertuples()):

            date = row.Index.to_pydatetime()
            high = row.high
            low = row.low
            close = row.close
            period_high = row.period_high
            end_flag = True if (i == len(self._ts) - 1) else False
            shares = 0

            if date < self._start:
                continue
            elif start_flag:
                start_flag = False
                # set start and end
                self._start = date
                self._end = self._ts.index[-1]

            # buy
            if (self._tlog.num_open_trades() == 0
                and high > period_high
                and not end_flag):

                # enter buy in trade log
                shares = self._tlog.enter_trade(date, close)
            # sell
            elif end_flag:

                # enter sell in trade log
                shares = self._tlog.exit_trade(date, close)

            if shares > 0:
                print("{0} BUY  {1} {2} @ {3:.2f}".format(
                      date, shares, self._symbol, close))
            elif shares < 0:
                print("{0} SELL {1} {2} @ {3:.2f}".format(
                      date, -shares, self._symbol, close))

            # record daily balance
            self._dbal.append(date, high, low, close,
                              self._tlog.shares, self._tlog.cash)

    def run(self):
        self._ts = pf.fetch_timeseries(self._symbol)
        self._ts = pf.select_tradeperiod(self._ts, self._start,
                                         self._end, use_adj=False)
        
        # Add technical indicator: X day high, and X day low
        period_high = pd.Series(self._ts.close).rolling(self._period).max()
        self._ts['period_high'] = period_high
        
        self._tlog = pf.TradeLog()
        self._dbal = pf.DailyBal()

        self._algo()

    def get_logs(self):
        """ return DataFrames """
        self.tlog = self._tlog.get_log()
        self.dbal = self._dbal.get_log(self.tlog)
        return self.tlog, self.dbal

Run Strategy

In [5]:
s = Strategy(symbol, capital, start, end, int(period))
s.run()
s._ts['2016-02-01':'2016-03-01']

2016-02-22 00:00:00 BUY  5 ^GSPC @ 1945.50
2019-12-23 00:00:00 SELL 5 ^GSPC @ 3224.01


,high,low,open,close,volume,adj_close,period_high
date,,,,,,,
2016-02-01,1947.20,1920.30,1936.94,1939.38,4322530000,1939.38,2016.71
2016-02-02,1935.26,1897.29,1935.26,1903.03,4463190000,1903.03,2016.71
2016-02-03,1918.01,1872.23,1907.07,1912.53,5172950000,1912.53,1990.26
2016-02-04,1927.35,1900.52,1911.67,1915.45,5193320000,1915.45,1943.09
2016-02-05,1913.07,1872.65,1913.07,1880.05,4929940000,1880.05,1940.24
2016-02-08,1873.25,1828.46,1873.25,1853.44,5636460000,1853.44,1940.24
2016-02-09,1868.25,1834.94,1848.46,1852.21,5183220000,1852.21,1940.24
2016-02-10,1881.60,1850.32,1857.10,1851.86,4471170000,1851.86,1940.24
2016-02-11,1847.00,1810.10,1847.00,1829.08,5500800000,1829.08,1940.24


Retrieve log DataFrames

In [6]:
tlog, dbal = s.get_logs()

In [7]:
tlog.tail()

,entry_date,entry_price,exit_date,exit_price,pl_points,pl_cash,qty,cumul_total
0,2016-02-22,1945.50,2019-12-23,3224.01,1278.51,6392.55,5,6392.55


The first 20 day high occurred on 2/22/2016, so we bought on the close that day.
We held onto the present, then sold the positon.